## What are Auckland's crashiest roads?

Use open data and GeoPandas to find out

In [ ]:
from pathlib import Path
import json

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import shapely.geometry as sg
import folium
import spectra

DATA_DIR = Path('../data')
OUT_DIR = Path('../output')

%matplotlib inline

## Download data

1. NZ crash data from NZTA at http://www.nzta.govt.nz/safety/safety-resources/road-safety-information-and-tools/disaggregated-crash-data/ 
2. Auckland road geodata from Mapzen at https://s3.amazonaws.com/metro-extracts.mapzen.com/auckland_new-zealand.imposm-geojson.zip; see also https://mapzen.com/data/metro-extracts/metro/auckland_new-zealand/


In [ ]:
%ls {DATA_DIR}

## Prepare data

In [ ]:
# Clean crash metadata
path = DATA_DIR/'disaggregated-crash-data-metadata.csv'
f = pd.read_csv(path, encoding='latin1')  # Ah, Windows

# Rename some
f = f.rename(columns={
    'Variable Name': 'variable',
    'Description': 'description',
  })

# Reformat some
f['variable'] = f['variable'].str.upper()
f = f[['variable', 'description']].copy()

path = DATA_DIR/'disaggregated-crash-data-metadata-cleaned.csv'
f.to_csv(path, index=False)

In [ ]:
# Filter crash data to last five years and save to (smaller) file
path = DATA_DIR/'disaggregated-crash-data.csv'
f = pd.read_csv(path)
f = f[f['CRASH_YEAR'] >= 2012].copy()

path = DATA_DIR/'nz_crashes_2012--2017.csv'
f.to_csv(path, index=False)


## Explore data

In [ ]:
# Load Auckland roads as GeoDataFrame

path = DATA_DIR/'auckland_new-zealand_roads_gen1.geojson'  # 50 m tolerance
roads = gpd.read_file(str(path))
roads['osm_id'] = roads['osm_id'].astype(int).astype(str)
roads.head()

In [ ]:
# Project to New Zealand Transverse Mercator (NZTM) coordinates

WGS84 = {'init': 'epsg:4326'}
NZTM = {'no_defs': True, 'init': 'epsg:2193'}

print('Is the current CRS WGS84?', roads.crs == WGS84)

roads = roads.to_crs(NZTM)
print('Is the new CRS NZTM?', roads.crs == NZTM)

roads.head()

In [ ]:
# Plot

f, ax = plt.subplots(1, figsize=(12, 12))
ax = roads.plot(column='type', ax=ax)
plt.show()


In [ ]:
# Load NZ crash data as a DataFrame

path = DATA_DIR/'nz_crashes_2012--2017.csv'
crashes = pd.read_csv(path)
crashes.head().T

In [ ]:
# Filter Auckland crashes and drop bad locations

f = crashes.copy()
cond = f['LG_REGION_DESC'].str.contains('Auckland')
cond &= (f['EASTING'] > 0) & (f['NORTHING'] > 0)
#cond &= f['MULTI_VEH'].str.contains(r'cyclist|pedestrian', case=False)
f = f[cond].copy()
f.T



In [ ]:
# What's CRASH_SEV?

path = DATA_DIR/'disaggregated-crash-data-metadata-cleaned.csv'
tmp = pd.read_csv(path)

cond = tmp['variable'] == 'CRASH_SEV'
tmp.loc[cond, 'description'].iat[0]

In [ ]:
# Convert to GeoDataFrame in NZTM

geometry = [sg.Point(p) for p in zip(f['EASTING'], f['NORTHING'])]
crashes = gpd.GeoDataFrame(f, crs=NZTM, geometry=geometry)
crashes.T

In [ ]:
# Plot crashes on roads

f, ax = plt.subplots(1, figsize=(12, 12))
base = roads.plot(color='black', ax=ax)
crashes.plot(ax=base, marker='o', color='red', markersize=3)
plt.show()

## Address our question

In [ ]:
# For each road collect the crashes near it

r = roads[['geometry', 'osm_id', 'name', 'class']].copy()

# Buffer crash points
buffer = 5  # meters
c = crashes[['geometry', 'CRASH_SEV']].copy()
c['geometry'] = c['geometry'].buffer(buffer)

# Spatial-join roads and buffered crash points 
f = gpd.sjoin(r, c, how='inner', op='intersects')
f

In [ ]:
# Assign a crash score to each road.
# Use the number of crashes per meter.
# Alternatively, could weight crashes by severity, but won't do that here.

f['num_crashes'] = 1
g = f.groupby('osm_id', as_index=False).agg({
  'num_crashes': 'sum', 
  'name': 'first',
  'geometry': 'first',
  })
g = gpd.GeoDataFrame(g, crs=NZTM)  # Lost GeoDataFrame during groupby :(

g['length'] = g.geometry.length
g['crash_score'] = g['num_crashes']/g['length']
g = g.sort_values('crash_score', ascending=False)

# Save to file for posterity
path = OUT_DIR/'auckland_crashy_roads.geojson'
with path.open('w') as tgt:
     tgt.write(g.to_crs(WGS84).to_json())

g

# Visualize the crashiest roads

In [ ]:
# Color-code roads by crash score with Spectra 

print(g.describe())

# Use Colorbrewer spectral colors
colors = reversed(['#d7191c', '#d7191c','#fdae61','#ffffbf','#abdda4','#2b83ba'])  
cuts = [0] + [g['crash_score'].quantile(k/100) for k in [25, 50, 75, 98, 100]]
scale = spectra.scale(colors).colorspace('lch').domain(cuts)
g['color'] = g['crash_score'].map(lambda x: scale(x).hexcode)

In [ ]:
# Plot on a slippy map with Folium

g = g.to_crs(WGS84) 
lon, lat = g.geometry.iat[0].coords[0]  # Center on crashiest road
tiles = 'https://cartodb-basemaps-{s}.global.ssl.fastly.net/light_all/{z}/{x}/{y}.png'
mappy = folium.Map(location=[lat, lon], zoom_start=16, tiles=tiles, attr='OSM')

def style(x):
    return {
        'color': x['properties']['color'],
        'weight': 5,
    }

geo = json.loads(g.sort_values('crash_score').to_json())
folium.GeoJson(geo, 
  style_function=style).add_to(mappy)

mappy


## Ideas for further exploration

- Use better resolution roads
- Do analysis on road name rather than road ID
- Choose a different metric?
- Forget roads and focus on areas; use a heatmap